# Sarsa & Saras($\lambda$)
> - Repeat(for each episode)
    - inital **s**    
    - Repeat(for each step in episode)   
        - choose **a** from **s** based on the policy Q(GREEDY-RATE $\epsilon$)        
        - Take action **a** get r, s'
        - *Choose action **a'** from **s'** based on the ploicy Q(GREEDY-RATE $\epsilon$) *
        - *Update $Q(s,a) \leftarrow Q(s,a) + \alpha[r+(\gamma Q(s',a') - Q(s,a))]$*
        - $s \leftarrow s'$
        - $a \leftarrow a'$
    - Until s is terminal
    
Sarsa 与普通Q-learning的区别是Q表的更新方式和动作选择上。

Q-learning得到下一个action和State的估计后会预测最大的值$max_{a'}Q(s',a')$来更新Q表，比较贪婪，在动作选择时却不一定选择$a' = argmax_{a'}Q(s',a')$ ；

而Sarsa的直接选择预测出的在做出a之后直接选择出下一阶段的a',并且执行，并根据选择的a'进行预测

In [1]:
# 物理环境env
import numpy as np
import time
import sys
if sys.version_info.major == 2:
    import Tkinter as tk
else:
    import tkinter as tk


UNIT = 40   # pixels
MAZE_H = 4  # grid height
MAZE_W = 4  # grid width


class Maze(tk.Tk, object):
    def __init__(self):
        super(Maze, self).__init__()
        self.action_space = ['u', 'd', 'l', 'r']
        self.n_actions = len(self.action_space)
        self.title('maze')
        self.geometry('{0}x{1}'.format(MAZE_H * UNIT, MAZE_H * UNIT))
        self._build_maze()

    def _build_maze(self):
        self.canvas = tk.Canvas(self, bg='white',
                           height=MAZE_H * UNIT,
                           width=MAZE_W * UNIT)

        # create grids
        for c in range(0, MAZE_W * UNIT, UNIT):
            x0, y0, x1, y1 = c, 0, c, MAZE_H * UNIT
            self.canvas.create_line(x0, y0, x1, y1)
        for r in range(0, MAZE_H * UNIT, UNIT):
            x0, y0, x1, y1 = 0, r, MAZE_H * UNIT, r
            self.canvas.create_line(x0, y0, x1, y1)

        # create origin
        origin = np.array([20, 20])

        # hell
        hell1_center = origin + np.array([UNIT * 2, UNIT])
        self.hell1 = self.canvas.create_rectangle(
            hell1_center[0] - 15, hell1_center[1] - 15,
            hell1_center[0] + 15, hell1_center[1] + 15,
            fill='black')
        # hell
        hell2_center = origin + np.array([UNIT, UNIT * 2])
        self.hell2 = self.canvas.create_rectangle(
            hell2_center[0] - 15, hell2_center[1] - 15,
            hell2_center[0] + 15, hell2_center[1] + 15,
            fill='black')

        # create oval
        oval_center = origin + UNIT * 2
        self.oval = self.canvas.create_oval(
            oval_center[0] - 15, oval_center[1] - 15,
            oval_center[0] + 15, oval_center[1] + 15,
            fill='yellow')

        # create red rect
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')

        # pack all
        self.canvas.pack()

    def reset(self):
        self.update()
        time.sleep(0.5)
        self.canvas.delete(self.rect)
        origin = np.array([20, 20])
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')
        # return observation
        return self.canvas.coords(self.rect)

    def step(self, action):
        s = self.canvas.coords(self.rect)
        base_action = np.array([0, 0])
        if action == 0:   # up
            if s[1] > UNIT:
                base_action[1] -= UNIT
        elif action == 1:   # down
            if s[1] < (MAZE_H - 1) * UNIT:
                base_action[1] += UNIT
        elif action == 2:   # right
            if s[0] < (MAZE_W - 1) * UNIT:
                base_action[0] += UNIT
        elif action == 3:   # left
            if s[0] > UNIT:
                base_action[0] -= UNIT

        self.canvas.move(self.rect, base_action[0], base_action[1])  # move agent

        s_ = self.canvas.coords(self.rect)  # next state

        # reward function
        if s_ == self.canvas.coords(self.oval):
            reward = 1
            done = True
            s_ = 'terminal'
        elif s_ in [self.canvas.coords(self.hell1), self.canvas.coords(self.hell2)]:
            reward = -1
            done = True
            s_ = 'terminal'
        else:
            reward = 0
            done = False

        return s_, reward, done

    def render(self):
        time.sleep(0.1)
        self.update()

In [44]:
# 思维决策
import pandas as pd
import numpy as np
class RL(object):
    def __init__(self,action_span,learning_rate = 0.9,reward_decay=0.1,e_greedy=0.9):
        self.actions = action_span
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy
        self.q_table = pd.DataFrame(columns = actions)
    def check_state_exist(self,observation):
        if observation not in self.q_table.index:
            self.q_table = self.q_table.append(
                pd.Series(index = self.q_table.columns,data = [0]*len(actions),name = observation)
            )
    def choose_action(self,observation):
        self.check_state_exist(observation)
        action_table = self.q_table.loc[observation,:]
        if np.random.uniform() < self.epsilon:
            action_table.reindex(np.random.permutation(action_table.index))
            action_chosen = action_table.idxmax()
        else:
            action_chosen = np.random.choice(actions)
        return action_chosen
    def learn(self):
        pass
rl = RL(['u','d','l','r'])

'u'

In [53]:
class Sarsa_table(RL):
    def __init__(self,action_span,learning_rate=0.1,reward_decay = 0.9,e_greedy = 0.9):
        super(Sarsa_table,self).__init__(action_span,learning_rate,reward_decay,e_greedy)
    def learn(self,S,a,r,S_,a_):
        self.check_state_exist(S_)
        q_predict = self.q_table.loc[S,a]
        if S_ !='terminal':
            q_target = r + self.gamma*self.q_table.loc[S_,a_]
        else:
            q_target = r
        self.q_table.loc.loc[S,a] += self.lr*(q_target - q_predict)         

TypeError: super() argument 1 must be type, not Sarsa_table

In [46]:
# 主程序
def update():
    for episode in range(100):
        # initial observation
        observation = env.reset()

        # RL choose action based on observation
        action = RL.choose_action(str(observation))

        while True:
            # fresh env
            env.render()

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            # RL choose action based on next observation
            action_ = RL.choose_action(str(observation_))

            # RL learn from this transition (s, a, r, s, a) ==> Sarsa
            RL.learn(str(observation), action, reward, str(observation_), action_)

            # swap observation and action
            observation = observation_
            action = action_

            # break while loop when end of this episode
            if done:
                break

    # end of game
    print('game over')
    env.destroy()

In [54]:
env = Maze()
RL = Sarsa_table(action_span=list(range(env.n_actions)))
env.after(100, update)
env.mainloop()

TypeError: 'Sarsa_table' object is not callable

## Sarsa($\lambda$)
与Sarsa(0)方法相比，有权重地进行单元更新。

![](https://morvanzhou.github.io/static/results/reinforcement-learning/3-3-1.png)

In [ ]:
class Sarsa_lambda_table(RL):
    def __init__(self,action_span,learning_rate=0.1,reward_decay = 0.9,e_greedy = 0.9,trace_decay = 0.9):
        super().__init__(action_span,learning_rate,reward_decay,e_greedy)
        self.lambda = trace_decay
        self.eligibility_table  = self.q_table.copy() #即算法步骤中的E(S,A)
    def check_state_exist(self,state):
        if state not in self.q_table.index:
            add_ = pd.Series(data = [0]*len(self.actions),index = self.q_table.clumns,name = state)
            self.q_table =  self.q_table.append(add_)
            self.eligibility_table =  self.eligibility_table.append(add_)
    def learn(self,S,a,r,S_,a_):
        self.check_state_exist(S_)
        q_predict = self.q_table.loc[S,a]
        if S_ != 'terminal':
            q_target = r + self.gamma*self.q_table.loc[S_,a_]
        else:
            q_target= r
        error = q_target - q_predict
        self.eligibility_table.loc[S,:] = 0
        self.eligibility_table.loc[S,a] = 1
        self.q_table += self.lr * self.eligibility_table * error
        self.eligibility_table *= self.lambda